#Install

In [ ]:
!pip install pandas

#Connection Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Read data


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
user_features_train = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/user_features_train.csv')

In [ ]:
course_features_train = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/course_features_train.csv')
train_log = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/train_log_processed.csv')

In [ ]:
course_info = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/course_info_processed.csv')

In [ ]:
user_info = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/user_info_processed.csv')

In [ ]:
test_log = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/test_log_processed.csv')

In [ ]:
test_truth = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/raw_data/test_truth.csv')

#user features

##Xử lý tập train để lấy previous_dropout_num tại các thời điểm

In [ ]:
#Xác định lần cuối cùng truy cập khóa học
latest_test_log = test_log.groupby('enroll_id')['time'].max().reset_index()
latest_test_log.columns = ['enroll_id','latest_date']
user_features_test = pd.merge(latest_test_log, test_log[['enroll_id','username']].drop_duplicates(), on='enroll_id', how='left')
user_features_test

,enroll_id,latest_date,username
0,775,2015-10-21,1520977
1,778,2015-10-30,671700
2,784,2015-10-10,1488662
3,788,2015-10-22,1068063
4,797,2015-09-26,1112353
...,...,...,...
67694,466770,2016-03-12,2670958
67695,466775,2016-03-05,2582621
67696,466777,2016-03-13,133215
67697,466783,2016-03-03,2665176


In [ ]:
#Xác định lần cuối cùng truy cập khóa học
latest_train_log = train_log.groupby('enroll_id')['time'].max().reset_index()
latest_train_log.columns = ['enroll_id','latest_date']
latest_train_log

,enroll_id,latest_date
0,772,2015-09-27
1,773,2015-10-19
2,774,2015-10-29
3,776,2015-10-25
4,777,2015-10-05
...,...,...
157938,466774,2016-03-27
157939,466776,2016-03-20
157940,466781,2016-04-02
157941,466782,2016-03-06


In [ ]:
user_train_log = pd.merge(latest_train_log, user_features_train, on='enroll_id', how='left')
user_train_log = user_train_log.drop(['Unnamed: 0'] , axis=1)
user_train_log = user_train_log.rename(columns={'latest_date':'old_latest_date'})

In [ ]:
def find_course_dropout(df, user_id, latest_date):
  # Lọc dữ liệu dựa trên điều kiện
  filtered_df = df[(df['user_id'] == user_id) & (df['old_latest_date'] < latest_date)]

  if filtered_df.empty:
    return 0
  else:
    prev_num = filtered_df.max()['prev_dropout_num']
    return prev_num

user_features_test['prev_dropout_num'] = user_features_test.apply(lambda row: find_course_dropout(user_train_log, row['username'], row['latest_date']), axis=1)
user_features_test

,enroll_id,old_latest_date,user_id,gender,education,age,prev_dropout_num
0,772,2015-09-27,5981,0,1,26.0,1
1,773,2015-10-19,1544995,1,0,18.0,1
2,774,2015-10-29,1072798,0,0,18.0,1
3,776,2015-10-25,561867,1,3,34.0,1
4,777,2015-10-05,1368125,0,0,18.0,1
...,...,...,...,...,...,...,...
157938,466774,2016-03-27,2588048,0,5,17.0,0
157939,466776,2016-03-20,2736225,0,1,19.0,1
157940,466781,2016-04-02,2830711,1,1,19.0,0
157941,466782,2016-03-06,2680742,1,2,19.0,0


## Education

In [ ]:
user_features_test = user_features_test.drop(['latest_date', 'education', 'gender'], axis=1)
user_features_test = pd.merge(user_features_test, user_info, on='user_id', how='left')
user_features_test

,enroll_id,user_id,prev_dropout_num,age,Unnamed: 0,gender,education,birth
0,775,1520977,0,18,9423369,1,Bachelor's,1997.0
1,778,671700,0,18,271050,1,Bachelor's,1997.0
2,784,1488662,1,18,4711979,1,Bachelor's,1997.0
3,788,1068063,0,17,2425352,0,High,1998.0
4,797,1112353,0,18,1697307,1,Bachelor's,1997.0
...,...,...,...,...,...,...,...,...
67694,466770,2670958,0,19,2709148,1,Bachelor's,1997.0
67695,466775,2582621,0,19,8003969,0,Bachelor's,1997.0
67696,466777,133215,0,16,7966499,0,Bachelor's,2000.0
67697,466783,2665176,0,19,585099,1,Bachelor's,1997.0


In [ ]:
education_map = {
    "Bachelor's": 0,
    "Master's": 1,
    "High": 2,
    "Associate": 3,
    "Doctorate": 4,
    "Middle": 5,
    "Primary": 6
}
user_features_test = user_features_test.drop(['Unnamed: 0'], axis=1)
user_features_test['education'] = user_features_test['education'].map(education_map)
user_features_test

,enroll_id,user_id,prev_dropout_num,age,gender,education,birth
0,775,1520977,0,18,1,0,1997.0
1,778,671700,0,18,1,0,1997.0
2,784,1488662,1,18,1,0,1997.0
3,788,1068063,0,17,0,2,1998.0
4,797,1112353,0,18,1,0,1997.0
...,...,...,...,...,...,...,...
67694,466770,2670958,0,19,1,0,1997.0
67695,466775,2582621,0,19,0,0,1997.0
67696,466777,133215,0,16,0,0,2000.0
67697,466783,2665176,0,19,1,0,1997.0


## Age

In [ ]:
latest_test_log = test_log.groupby('enroll_id')['time'].max().reset_index()
latest_test_log.columns = ['enroll_id','latest_date']
user_features_test = pd.merge(user_features_test,latest_test_log,on='enroll_id')
user_features_test

,enroll_id,user_id,prev_dropout_num,age,gender,education,birth,latest_date
0,775,1520977,0,18,1,0,1997.0,2015-10-21
1,778,671700,0,18,1,0,1997.0,2015-10-30
2,784,1488662,1,18,1,0,1997.0,2015-10-10
3,788,1068063,0,17,0,2,1998.0,2015-10-22
4,797,1112353,0,18,1,0,1997.0,2015-09-26
...,...,...,...,...,...,...,...,...
67694,466770,2670958,0,19,1,0,1997.0,2016-03-12
67695,466775,2582621,0,19,0,0,1997.0,2016-03-05
67696,466777,133215,0,16,0,0,2000.0,2016-03-13
67697,466783,2665176,0,19,1,0,1997.0,2016-03-03


In [ ]:

def tinh_tuoi(row):
    row = pd.to_datetime(row)
    tuoi = row.dt.year
    return tuoi

user_features_test['age'] = tinh_tuoi(user_features_test['latest_date']) - user_features_test['birth']
user_features_test = user_features_test.drop('birth', axis=1)
user_features_test['age'] = user_features_test['age'].astype(int)
user_features_test

,enroll_id,user_id,prev_dropout_num,age,gender,education,latest_date
0,775,1520977,0,18,1,0,2015-10-21
1,778,671700,0,18,1,0,2015-10-30
2,784,1488662,1,18,1,0,2015-10-10
3,788,1068063,0,17,0,2,2015-10-22
4,797,1112353,0,18,1,0,2015-09-26
...,...,...,...,...,...,...,...
67694,466770,2670958,0,19,1,0,2016-03-12
67695,466775,2582621,0,19,0,0,2016-03-05
67696,466777,133215,0,16,0,0,2016-03-13
67697,466783,2665176,0,19,1,0,2016-03-03


In [ ]:
check = user_features_test[user_features_test['prev_dropout_num'] > 2]
check

,enroll_id,user_id,prev_dropout_num,age,gender,education,latest_date
8,814,180034,6,20,0,2,2015-10-23
21,868,842580,5,18,0,2,2015-10-24
24,882,6233,11,32,0,0,2015-09-26
32,917,16024,3,18,0,1,2015-09-28
53,1010,3664,12,43,0,3,2015-10-20
...,...,...,...,...,...,...,...
67630,466432,491129,26,29,0,0,2016-03-13
67653,466549,2803111,5,19,1,2,2016-04-01
67658,466588,109766,6,25,0,1,2016-03-14
67662,466605,458193,15,21,1,0,2016-03-12


In [ ]:
user_features_test = user_features_test.drop(['latest_date'], axis=1)
user_features_test = user_features_test.rename(columns={'username':'user_id'})
user_features_test

,enroll_id,user_id,prev_dropout_num,age,gender,education
0,775,1520977,0,18,1,0
1,778,671700,0,18,1,0
2,784,1488662,1,18,1,0
3,788,1068063,0,17,0,2
4,797,1112353,0,18,1,0
...,...,...,...,...,...,...
67694,466770,2670958,0,19,1,0
67695,466775,2582621,0,19,0,0
67696,466777,133215,0,16,0,0
67697,466783,2665176,0,19,1,0


In [ ]:
user_features_test.to_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/user_features_test.csv')

#Course features

##Số lượng học viên đã tham gia vào khóa học

In [ ]:
#Xác định lần cuối cùng truy cập khóa học
latest_test_log = test_log.groupby('enroll_id')['time'].max().reset_index()
latest_test_log.columns = ['enroll_id','latest_date']
course_features_test = pd.merge(latest_test_log, test_log[['enroll_id','course_id']].drop_duplicates(), on='enroll_id', how='left')
course_features_test

,enroll_id,latest_date,course_id
0,775,2015-10-21,course-v1:TsinghuaX+70800232X+2015_T2
1,778,2015-10-30,course-v1:TsinghuaX+70800232X+2015_T2
2,784,2015-10-10,course-v1:TsinghuaX+70800232X+2015_T2
3,788,2015-10-22,course-v1:TsinghuaX+70800232X+2015_T2
4,797,2015-09-26,course-v1:TsinghuaX+70800232X+2015_T2
...,...,...,...
67694,466770,2016-03-12,course-v1:TsinghuaX+AP000001X+2016_T1
67695,466775,2016-03-05,course-v1:TsinghuaX+AP000001X+2016_T1
67696,466777,2016-03-13,course-v1:TsinghuaX+AP000001X+2016_T1
67697,466783,2016-03-03,course-v1:TsinghuaX+AP000001X+2016_T1


In [ ]:
#Xác định lần cuối cùng truy cập khóa học
latest_train_log = train_log.groupby('enroll_id')['time'].max().reset_index()
latest_train_log.columns = ['enroll_id','latest_date']
latest_train_log

,enroll_id,latest_date
0,772,2015-09-27
1,773,2015-10-19
2,774,2015-10-29
3,776,2015-10-25
4,777,2015-10-05
...,...,...
157938,466774,2016-03-27
157939,466776,2016-03-20
157940,466781,2016-04-02
157941,466782,2016-03-06


In [ ]:
latest_train_log = pd.merge(latest_train_log, course_features_train, on='enroll_id', how='left')
latest_train_log = latest_train_log.drop(['Unnamed: 0'] , axis=1)
latest_train_log = latest_train_log.rename(columns={'latest_date':'old_latest_date'})
latest_train_log

,enroll_id,old_latest_date,course_id,previous_enroll_num,previous_dropout_user_num,duration
0,772,2015-09-27,course-v1:TsinghuaX+70800232X+2015_T2,25,25,103
1,773,2015-10-19,course-v1:TsinghuaX+70800232X+2015_T2,561,523,103
2,774,2015-10-29,course-v1:TsinghuaX+70800232X+2015_T2,850,711,103
3,776,2015-10-25,course-v1:TsinghuaX+70800232X+2015_T2,716,637,103
4,777,2015-10-05,course-v1:TsinghuaX+70800232X+2015_T2,223,212,103
...,...,...,...,...,...,...
157938,466774,2016-03-27,course-v1:TsinghuaX+AP000001X+2016_T1,124,111,120
157939,466776,2016-03-20,course-v1:TsinghuaX+AP000001X+2016_T1,92,84,120
157940,466781,2016-04-02,course-v1:TsinghuaX+AP000001X+2016_T1,193,163,120
157941,466782,2016-03-06,course-v1:TsinghuaX+AP000001X+2016_T1,23,21,120


In [ ]:
def find_dropout(df, course_id, latest_date):
  # Lọc dữ liệu dựa trên điều kiện
  filtered_df = df[(df['course_id'] == course_id) & (df['old_latest_date'] < latest_date)]

  if filtered_df.empty:
    return 0
  else:
    prev_num = filtered_df.max()['previous_enroll_num']
    return prev_num

course_features_test['previous_enroll_num'] = course_features_test.apply(lambda row: find_dropout(latest_train_log, row['course_id'], row['latest_date']), axis=1)
course_features_test

,enroll_id,latest_date,course_id,previous_enroll_num
0,775,2015-10-21,course-v1:TsinghuaX+70800232X+2015_T2,610
1,778,2015-10-30,course-v1:TsinghuaX+70800232X+2015_T2,880
2,784,2015-10-10,course-v1:TsinghuaX+70800232X+2015_T2,344
3,788,2015-10-22,course-v1:TsinghuaX+70800232X+2015_T2,631
4,797,2015-09-26,course-v1:TsinghuaX+70800232X+2015_T2,0
...,...,...,...,...
67694,466770,2016-03-12,course-v1:TsinghuaX+AP000001X+2016_T1,42
67695,466775,2016-03-05,course-v1:TsinghuaX+AP000001X+2016_T1,8
67696,466777,2016-03-13,course-v1:TsinghuaX+AP000001X+2016_T1,45
67697,466783,2016-03-03,course-v1:TsinghuaX+AP000001X+2016_T1,4


## Số lượng học viên đã bỏ học

In [ ]:
def find_dropout(df, course_id, latest_date):
  # Lọc dữ liệu dựa trên điều kiện
  filtered_df = df[(df['course_id'] == course_id) & (df['old_latest_date'] < latest_date)]

  if filtered_df.empty:
    return 0
  else:
    prev_num = filtered_df.max()['previous_dropout_user_num']
    return prev_num

course_features_test['previous_dropout_user_num'] = course_features_test.apply(lambda row: find_dropout(lastest_train_log, row['course_id'], row['latest_date']), axis=1)
course_features_test

,enroll_id,latest_date,course_id,previous_enroll_num,previous_dropout_user_num
0,775,2015-10-21,course-v1:TsinghuaX+70800232X+2015_T2,610,563
1,778,2015-10-30,course-v1:TsinghuaX+70800232X+2015_T2,880,729
2,784,2015-10-10,course-v1:TsinghuaX+70800232X+2015_T2,344,328
3,788,2015-10-22,course-v1:TsinghuaX+70800232X+2015_T2,631,582
4,797,2015-09-26,course-v1:TsinghuaX+70800232X+2015_T2,0,0
...,...,...,...,...,...
67694,466770,2016-03-12,course-v1:TsinghuaX+AP000001X+2016_T1,42,40
67695,466775,2016-03-05,course-v1:TsinghuaX+AP000001X+2016_T1,8,7
67696,466777,2016-03-13,course-v1:TsinghuaX+AP000001X+2016_T1,45,42
67697,466783,2016-03-03,course-v1:TsinghuaX+AP000001X+2016_T1,4,4


## Duration

In [ ]:
course_features_test = pd.merge(course_features_test, latest_train_log[['enroll_id','duration']], on='enroll_id', how='left')
course_features_test['duration'] = course_info['duration'].astype(int)
course_features_test = course_features_test.fillna(0)
course_features_test = course_features_test.drop(['Unnamed: 0'], axis=1)
course_features_test

,enroll_id,latest_date,course_id,previous_enroll_num,previous_dropout_user_num,duration
0,775,2015-10-21,course-v1:TsinghuaX+70800232X+2015_T2,610,563,125.0
1,778,2015-10-30,course-v1:TsinghuaX+70800232X+2015_T2,880,729,324.0
2,784,2015-10-10,course-v1:TsinghuaX+70800232X+2015_T2,344,328,1089.0
3,788,2015-10-22,course-v1:TsinghuaX+70800232X+2015_T2,631,582,58.0
4,797,2015-09-26,course-v1:TsinghuaX+70800232X+2015_T2,0,0,96.0
...,...,...,...,...,...,...
67694,466770,2016-03-12,course-v1:TsinghuaX+AP000001X+2016_T1,42,40,NaN
67695,466775,2016-03-05,course-v1:TsinghuaX+AP000001X+2016_T1,8,7,NaN
67696,466777,2016-03-13,course-v1:TsinghuaX+AP000001X+2016_T1,45,42,NaN
67697,466783,2016-03-03,course-v1:TsinghuaX+AP000001X+2016_T1,4,4,NaN


In [ ]:
course_features_test.to_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/course_features_test.csv')

# Log features

## Tổng số session

In [ ]:
# Đếm số lượng session_id
session_num_test = test_log.groupby('enroll_id')['session_id'].nunique().reset_index()
session_num_test.columns = ['enroll_id', 'session_num']
session_num_test

,enroll_id,session_num
0,775,2
1,778,2
2,784,2
3,788,1
4,797,1
...,...,...
67694,466770,2
67695,466775,2
67696,466777,2
67697,466783,1


## Tổng số lần tương tác với video

In [ ]:
#Lọc ra các action có từ 'video'
video_actions_test = test_log[test_log['action'].str.contains('video')]
# Lấy ra số lần tương tác video theo từng enroll_id
video_num_test = video_actions_test.groupby('enroll_id').size().reset_index(name='video_num')

# Merge lại với enroll_id của test_log để lấy ra đc cái user không có action video và set giá trị bằng 0
enroll_ids_test = test_log[['enroll_id']].drop_duplicates()
video_num_test = pd.merge(enroll_ids_test, video_num_test, on='enroll_id', how='left').fillna(0)

video_num_test['video_num'] = video_num_test['video_num'].astype(int)
print(video_num_test)

       enroll_id  video_num
0            775         30
1            778        918
2            784          3
3            788          0
4            797         22
...          ...        ...
67694     466770         33
67695     466775          0
67696     466777          6
67697     466783          0
67698     466785          9

[67699 rows x 2 columns]


## Tổng số lần tương tác với giáo trình

In [ ]:
#Lọc ra các action có từ 'courseware'
courseware_actions_test = test_log[test_log['action'].str.contains('courseware')]
# Lấy ra số lượng coi courseware theo từng enroll_id
courseware_num_test = courseware_actions_test.groupby('enroll_id').size().reset_index(name='courseware_num')

# Merge lại với enroll_id của test_log để lấy ra đc cái user không có action courseware và set giá trị bằng 0
courseware_num_test = pd.merge(enroll_ids_test, courseware_num_test, on='enroll_id', how='left').fillna(0)
courseware_num_test['courseware_num'] = courseware_num_test['courseware_num'].astype(int)

print(courseware_num_test)

       enroll_id  courseware_num
0            775               9
1            778             282
2            784               6
3            788               0
4            797               2
...          ...             ...
67694     466770               9
67695     466775               2
67696     466777               6
67697     466783               0
67698     466785               6

[67699 rows x 2 columns]


In [ ]:
log_features_test = pd.merge(session_num_test, video_num_test, on='enroll_id', how='inner')
log_features_test = pd.merge(log_features_test, courseware_num_test, on='enroll_id', how='inner')
log_features_test.to_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/log_features_test.csv')

In [ ]:
course_features_test = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/course_features_test.csv')
course_features_test = course_features_test.drop(['Unnamed: 0'], axis=1)
log_features_test = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/log_features_test.csv')
log_features_test = log_features_test.drop(['Unnamed: 0'], axis=1)
user_features_test = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/user_features_test.csv')
user_features_test = user_features_test.drop(['Unnamed: 0'], axis=1)

In [ ]:
print(len(user_features_test))
print(len(course_features_test))
print(len(log_features_test))

67699
67699
67699


In [ ]:
test_features = pd.merge(user_features_test, course_features_test, on='enroll_id', how='inner')
test_features = pd.merge(test_features, log_features_test, on='enroll_id', how='inner')
test_features = pd.merge(test_features, test_truth, on='enroll_id', how='inner')
test_features = test_features.drop(['course_id','latest_date','gender'], axis=1)
test_features

,enroll_id,user_id,prev_dropout_num,age,education,previous_enroll_num,previous_dropout_user_num,duration,session_num,video_num,courseware_num,truth
0,775,1520977,0,18,0,610,563,125.0,2,30,9,1
1,778,671700,0,18,0,880,729,324.0,2,918,282,0
2,784,1488662,1,18,0,344,328,1089.0,2,3,6,1
3,788,1068063,0,17,2,631,582,58.0,1,0,0,1
4,797,1112353,0,18,0,0,0,96.0,1,22,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
67694,466770,2670958,0,19,0,42,40,NaN,2,33,9,1
67695,466775,2582621,0,19,0,8,7,NaN,2,0,2,1
67696,466777,133215,0,16,0,45,42,NaN,2,6,6,0
67697,466783,2665176,0,19,0,4,4,NaN,1,0,0,1


In [ ]:
test_features = test_features.drop(['user_id'], axis=1)
test_features.to_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/test_set.csv')

In [ ]:
test_features.to_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/test_set.csv')

In [ ]:
test_features = pd.read_csv('/content/drive/MyDrive/Nhóm 11/Đồ án môn học/Technology/data_sau_xử_lý/trích_xuất_dữ_liệu/test_set.csv')

In [ ]:
x = test_truth['enroll_id']

In [ ]:
x

,enroll_id
0,775
1,778
2,784
3,788
4,797
...,...
67694,466770
67695,466775
67696,466777
67697,466783


In [ ]:

y = test_log[['enroll_id', 'course_id']].drop_duplicates()
y


,enroll_id,course_id
0,775,course-v1:TsinghuaX+70800232X+2015_T2
39,778,course-v1:TsinghuaX+70800232X+2015_T2
1277,784,course-v1:TsinghuaX+70800232X+2015_T2
1289,788,course-v1:TsinghuaX+70800232X+2015_T2
1292,797,course-v1:TsinghuaX+70800232X+2015_T2
...,...,...
12944714,466770,course-v1:TsinghuaX+AP000001X+2016_T1
12944762,466775,course-v1:TsinghuaX+AP000001X+2016_T1
12944781,466777,course-v1:TsinghuaX+AP000001X+2016_T1
12944810,466783,course-v1:TsinghuaX+AP000001X+2016_T1


In [ ]:
y = pd.merge(y, course_info[['course_id', 'duration']], on='course_id', how='inner')
y

,enroll_id,course_id,duration
0,775,course-v1:TsinghuaX+70800232X+2015_T2,103
1,778,course-v1:TsinghuaX+70800232X+2015_T2,103
2,784,course-v1:TsinghuaX+70800232X+2015_T2,103
3,788,course-v1:TsinghuaX+70800232X+2015_T2,103
4,797,course-v1:TsinghuaX+70800232X+2015_T2,103
...,...,...,...
67694,466770,course-v1:TsinghuaX+AP000001X+2016_T1,120
67695,466775,course-v1:TsinghuaX+AP000001X+2016_T1,120
67696,466777,course-v1:TsinghuaX+AP000001X+2016_T1,120
67697,466783,course-v1:TsinghuaX+AP000001X+2016_T1,120


In [ ]:
test_features = test_features.drop(['duration'], axis=1)
test_features

,enroll_id,prev_dropout_num,age,education,previous_enroll_num,previous_dropout_user_num,session_num,video_num,courseware_num,truth
0,775,0,18,0,610,563,2,30,9,1
1,778,0,18,0,880,729,2,918,282,0
2,784,1,18,0,344,328,2,3,6,1
3,788,0,17,2,631,582,1,0,0,1
4,797,0,18,0,0,0,1,22,2,1
...,...,...,...,...,...,...,...,...,...,...
67694,466770,0,19,0,42,40,2,33,9,1
67695,466775,0,19,0,8,7,2,0,2,1
67696,466777,0,16,0,45,42,2,6,6,0
67697,466783,0,19,0,4,4,1,0,0,1


In [ ]:
test_features = pd.merge(test_features, y[['enroll_id','duration']], on='enroll_id', how='inner')
test_features

,enroll_id,prev_dropout_num,age,education,previous_enroll_num,previous_dropout_user_num,session_num,video_num,courseware_num,truth,duration
0,775,0,18,0,610,563,2,30,9,1,103
1,778,0,18,0,880,729,2,918,282,0,103
2,784,1,18,0,344,328,2,3,6,1,103
3,788,0,17,2,631,582,1,0,0,1,103
4,797,0,18,0,0,0,1,22,2,1,103
...,...,...,...,...,...,...,...,...,...,...,...
67694,466770,0,19,0,42,40,2,33,9,1,120
67695,466775,0,19,0,8,7,2,0,2,1,120
67696,466777,0,16,0,45,42,2,6,6,0,120
67697,466783,0,19,0,4,4,1,0,0,1,120


In [ ]:
test_features.isnull().sum()

,0
enroll_id,0
prev_dropout_num,0
age,0
education,0
previous_enroll_num,0
previous_dropout_user_num,0
session_num,0
video_num,0
courseware_num,0
truth,0
